# Priority queue with heap

Develop a class `MyFirstPriorityQueue` that implements a heap-based priority queue. 


In [ ]:
class MyFirstPriorityQueue:
    """A heap-based priority queue implementation."""

    def __init__(self):